https://python.langchain.com/docs/use_cases/question_answering/quickstart

pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

api_key = os.environ['OPENAI_API_KEY']

In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
type(docs[0])

In [ ]:
# calling the chain 
rag_chain.invoke("What is Task Decomposition?")

In [ ]:
# retrieval with source
from langchain_core.runnables import RunnableParallel

# this mini sub chain extracts context from the context/question dict
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# runs the commands to retrieve context and pass through the question in parallel
# the resulting dict is assigned a new field and runnable, which is invoked when user asks a question.
# doing it this way preserves context by just keeping the entire dict.
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("What is Task Decomposition")

In [ ]:
# to add chat history to the context, a subchain is required to recontextualize the question. 
# this is due to the way that the LLM ingests the history. the q/a should be stand-alone at the end.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [ ]:
# building the QA chain with chat history

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
chat_history = []

question1 = "What is Task Decomposition?"
ai_msg1 = rag_chain.invoke({"question": question1, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question1), ai_msg1])

question2 = "What are common ways of doing it?"
ai_msg2 = rag_chain.invoke({"question": question2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question2), ai_msg2])

In [ ]:
chat_history